### CNN Model and Optmization of it using Keras_Tuner

In [19]:
!pip install keras-tuner==1.0.0

     |████████████████████████████████| 92kB 7.4MB/s 
  Found existing installation: keras-tuner 1.0.2
    Uninstalling keras-tuner-1.0.2:
      Successfully uninstalled keras-tuner-1.0.2


In [20]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.4.0


### We will use Fashion MNIST Dataset to create CNN classification model

These dataset contains various images related to fashion such as shirt,pants,shoe,bag etc

In [21]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [22]:
type(train_images)

numpy.ndarray

In [23]:
train_images.shape

(60000, 28, 28)

In [24]:
train_labels.shape

(60000,)

In [25]:
test_images.shape , test_labels.shape

((10000, 28, 28), (10000,))

In [26]:
train_images[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
          0,   0,  13,  73,   0,   0,   1,   4,   0,   0,   0,   0,   1,
          1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          0,  36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,
          0,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,
          0, 102, 204, 176, 134, 144, 123,  23,   0,   0,   0,   0,  12,
         10,   0],
       [  

In [27]:
train_images[0].shape

(28, 28)

### Scaling of  each pixel between 0-1 

In [28]:
train_images=train_images/255.0
test_images=test_images/255.0

train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

### Now in CNN we have to give input in following shape :

1. Total No of images
2. Length of Pixels in Image
3. Width of Pixels in Image
4. Black and White (1 channel)

In [29]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1) 

train_images.shape , test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

### Now we will create a CNN model function along with Optimizer

These optimizer will help us to select no of Convolution/Max pooling layers

1. First we will add Sequential Layer architecture that we want
2. Now we will add Convolution layer -- Which contains no of filters,kernel_size or filter_size , activation function , input shape 

Now these will help us to find which no of filters are better , which kernel size is better

In [30]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [32]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output1',project_name="Mnist Fashion")

tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.9036666750907898

Best val_accuracy So Far: 0.909500002861023
Total elapsed time: 00h 01m 44s
INFO:tensorflow:Oracle triggered exit


In [34]:
tuner_search.results_summary()

Results summary
Results in output1/Mnist Fashion
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 80
learning_rate: 0.001
Score: 0.909500002861023
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 5
dense_1_units: 48
learning_rate: 0.001
Score: 0.9039999842643738
Trial summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 3
dense_1_units: 112
learning_rate: 0.001
Score: 0.9036666750907898
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 112
learning_rate: 0.01
Score: 0.8826666474342346
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 3
dense_1_units: 80
learning_rate: 0.01
Score: 0.8806666731834412


In [35]:
tuner_search.get_best_models(num_models=1)[0]

In [36]:
model=tuner_search.get_best_models(num_models=1)[0]

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        38432     
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1024080   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,064,570
Trainable params: 1,064,570
Non-trainable params: 0
_________________________________________________________________


In [37]:
 model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1715 - accuracy: 0.9364 - val_loss: 0.2545 - val_accuracy: 0.9140
Epoch 5/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1350 - accuracy: 0.9501 - val_loss: 0.2810 - val_accuracy: 0.9078
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1063 - accuracy: 0.9613 - val_loss: 0.2834 - val_accuracy: 0.9125
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0798 - accuracy: 0.9705 - val_loss: 0.3274 - val_accuracy: 0.9062
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0643 - accuracy: 0.9769 - val_loss: 0.3443 - val_accuracy: 0.9037
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0550 - accuracy: 0.9792 - val_loss: 0.3734 - val_accuracy: 0.9137
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0395 - accuracy: 0.9863 - val_loss: 0.3894 - val_accuracy